# 🎤 Piper TTS - Training Completo da Zero

## 📋 Panoramica
Questo notebook permette di addestrare un modello Piper TTS completamente da zero.

**Differenze con Fine-Tuning:**
- ✅ Training completo: crei un nuovo modello da zero
- ✅ Massimo controllo su architettura e hyperparameters
- ✅ Ideale per lingue/accenti non supportati
- ⏱️ Tempo: ~12-16 ore (vs 8-12h del fine-tuning)

**Requisiti:**
- Google Colab con GPU (T4 o superiore)
- Dataset audio pulito (min 30 minuti, consigliato 2+ ore)
- File di trascrizione accurati

---

## 🔧 Step 1: Setup Ambiente

In [ ]:
# Verifica GPU
!nvidia-smi

print("\n✅ Se vedi info GPU sopra, sei pronto!")
print("❌ Se errore: Runtime > Change runtime type > GPU")

In [ ]:
# Installazione dipendenze
print("📦 Installazione Piper Training...")

!pip install -q piper-phonemize
!pip install -q onnxruntime
!pip install -q espeak-ng

# Clona repository Piper
!git clone https://github.com/rhasspy/piper.git
%cd piper/src/python

# Installa requirements
!pip install -q -r requirements.txt
!pip install -q -e .

%cd /content
print("\n✅ Installazione completata!")

## 📁 Step 2: Preparazione Dataset

### Struttura Dataset Richiesta:
```
my_dataset/
├── wavs/              # File audio WAV (16kHz, mono)
│   ├── audio_001.wav
│   ├── audio_002.wav
│   └── ...
└── metadata.csv       # Trascrizioni
```

### Formato metadata.csv:
```
audio_001|Questa è la prima frase.
audio_002|Questa è la seconda frase.
```

In [ ]:
# Monta Google Drive (se il dataset è lì)
from google.colab import drive
drive.mount('/content/drive')

# Oppure carica manualmente i file
# from google.colab import files
# uploaded = files.upload()

In [ ]:
# Configura percorsi
DATASET_DIR = "/content/drive/MyDrive/my_dataset"  # ⚠️ MODIFICA QUI
OUTPUT_DIR = "/content/piper_output"

!mkdir -p {OUTPUT_DIR}

# Verifica dataset
import os

assert os.path.exists(DATASET_DIR), f"❌ Dataset non trovato in {DATASET_DIR}"
assert os.path.exists(f"{DATASET_DIR}/metadata.csv"), "❌ metadata.csv mancante"
assert os.path.exists(f"{DATASET_DIR}/wavs"), "❌ Cartella wavs/ mancante"

# Conta file audio
num_wavs = len([f for f in os.listdir(f"{DATASET_DIR}/wavs") if f.endswith('.wav')])
print(f"\n✅ Dataset trovato: {num_wavs} file audio")

if num_wavs < 100:
    print("⚠️ Warning: Pochi file audio. Consigliati almeno 500-1000 per buoni risultati.")

## 🔍 Step 3: Validazione Dataset

In [ ]:
# Verifica formato audio
import librosa
import pandas as pd

# Carica metadata
metadata = pd.read_csv(f"{DATASET_DIR}/metadata.csv", 
                       sep='|', 
                       header=None, 
                       names=['filename', 'text'])

print(f"📊 Statistiche Dataset:")
print(f"   Totale frasi: {len(metadata)}")
print(f"   Lunghezza media testo: {metadata['text'].str.len().mean():.0f} caratteri")

# Verifica alcuni file audio
sample_files = metadata['filename'].head(5).tolist()

for fname in sample_files:
    wav_path = f"{DATASET_DIR}/wavs/{fname}.wav"
    if os.path.exists(wav_path):
        y, sr = librosa.load(wav_path, sr=None)
        duration = len(y) / sr
        print(f"   {fname}: {sr}Hz, {duration:.2f}s")
        
        if sr != 16000:
            print(f"      ⚠️ Sample rate non ottimale. Consigliato: 16000Hz")
    else:
        print(f"   ❌ File mancante: {wav_path}")

print("\n✅ Validazione completata!")

In [ ]:
# Verifica COMPLETA del metadata.csv
import pandas as pd
import re
from pathlib import Path

print("🔍 Verifica metadata.csv in corso...\n")

metadata_path = f"{DATASET_DIR}/metadata.csv"
wavs_dir = Path(f"{DATASET_DIR}/wavs")

# Carica metadata
try:
    metadata = pd.read_csv(metadata_path, sep='|', header=None, names=['filename', 'text'])
    print(f"✅ File caricato: {len(metadata)} righe\n")
except Exception as e:
    print(f"❌ ERRORE nel caricamento: {e}")
    raise

# === VERIFICA 1: Formato righe ===
print("="*60)
print("1️⃣ VERIFICA FORMATO RIGHE")
print("="*60)

invalid_rows = []
for idx, row in metadata.iterrows():
    # Verifica che ci siano esattamente 2 colonne
    if pd.isna(row['filename']) or pd.isna(row['text']):
        invalid_rows.append(f"Riga {idx+1}: Manca filename o text")
    # Verifica che filename non contenga caratteri strani
    elif not re.match(r'^[a-zA-Z0-9_-]+$', str(row['filename'])):
        invalid_rows.append(f"Riga {idx+1}: Filename '{row['filename']}' contiene caratteri non validi")
    # Verifica che text non sia vuoto
    elif len(str(row['text']).strip()) == 0:
        invalid_rows.append(f"Riga {idx+1}: Testo vuoto per '{row['filename']}'")

if invalid_rows:
    print(f"❌ Trovate {len(invalid_rows)} righe con formato non valido:")
    for err in invalid_rows[:5]:
        print(f"   • {err}")
    if len(invalid_rows) > 5:
        print(f"   ... e altre {len(invalid_rows)-5} righe")
else:
    print("✅ Tutte le righe hanno formato corretto")

# === VERIFICA 2: Corrispondenza con file WAV ===
print("\n" + "="*60)
print("2️⃣ VERIFICA CORRISPONDENZA FILE WAV")
print("="*60)

# File menzionati in metadata
metadata_files = set(metadata['filename'].astype(str))
# File realmente presenti (senza estensione)
actual_files = set(f.stem for f in wavs_dir.glob("*.wav"))

# File in metadata ma non in wavs/
missing_wavs = metadata_files - actual_files
# File in wavs/ ma non in metadata
extra_wavs = actual_files - metadata_files

print(f"   File in metadata: {len(metadata_files)}")
print(f"   File in wavs/: {len(actual_files)}")

if missing_wavs:
    print(f"\n❌ {len(missing_wavs)} file citati in metadata ma MANCANTI in wavs/:")
    for f in list(missing_wavs)[:5]:
        print(f"   • {f}.wav")
    if len(missing_wavs) > 5:
        print(f"   ... e altri {len(missing_wavs)-5} file")

if extra_wavs:
    print(f"\n⚠️ {len(extra_wavs)} file in wavs/ ma NON citati in metadata:")
    for f in list(extra_wavs)[:5]:
        print(f"   • {f}.wav")
    if len(extra_wavs) > 5:
        print(f"   ... e altri {len(extra_wavs)-5} file")

if not missing_wavs and not extra_wavs:
    print("\n✅ Perfetta corrispondenza! Tutti i file sono allineati.")

# === VERIFICA 3: Statistiche testo ===
print("\n" + "="*60)
print("3️⃣ STATISTICHE TRASCRIZIONI")
print("="*60)

text_lengths = metadata['text'].str.len()
print(f"   Lunghezza media: {text_lengths.mean():.1f} caratteri")
print(f"   Lunghezza min: {text_lengths.min()} caratteri")
print(f"   Lunghezza max: {text_lengths.max()} caratteri")

# Verifica trascrizioni troppo corte o troppo lunghe
too_short = metadata[text_lengths < 10]
too_long = metadata[text_lengths > 500]

if len(too_short) > 0:
    print(f"\n⚠️ {len(too_short)} trascrizioni troppo corte (<10 caratteri):")
    for _, row in too_short.head(3).iterrows():
        print(f"   • {row['filename']}: '{row['text']}'")

if len(too_long) > 0:
    print(f"\n⚠️ {len(too_long)} trascrizioni molto lunghe (>500 caratteri):")
    for _, row in too_long.head(3).iterrows():
        print(f"   • {row['filename']}: {len(row['text'])} caratteri")

# === VERIFICA 4: Duplicati ===
print("\n" + "="*60)
print("4️⃣ VERIFICA DUPLICATI")
print("="*60)

duplicate_files = metadata[metadata.duplicated(subset=['filename'], keep=False)]
if len(duplicate_files) > 0:
    print(f"❌ Trovati {len(duplicate_files)} filename duplicati:")
    print(duplicate_files[['filename']].drop_duplicates())
else:
    print("✅ Nessun filename duplicato")

# === RIEPILOGO FINALE ===
print("\n" + "="*60)
print("📊 RIEPILOGO FINALE")
print("="*60)

issues_count = len(invalid_rows) + len(missing_wavs) + len(duplicate_files)

if issues_count == 0 and len(extra_wavs) == 0:
    print("🎉 TUTTO PERFETTO! Il metadata.csv è correttamente formattato!")
    print(f"   ✓ {len(metadata)} righe valide")
    print(f"   ✓ Corrispondenza perfetta con i file WAV")
    print(f"   ✓ Nessun duplicato")
    print("\n✅ Sei pronto per il training!")
else:
    print(f"⚠️ Trovati {issues_count} problemi che DEVONO essere risolti:")
    if invalid_rows:
        print(f"   • {len(invalid_rows)} righe con formato errato")
    if missing_wavs:
        print(f"   • {len(missing_wavs)} file WAV mancanti")
    if duplicate_files:
        print(f"   • {len(duplicate_files)} filename duplicati")
    if extra_wavs:
        print(f"\n⚠️ Nota: {len(extra_wavs)} file WAV extra (non bloccante)")
    
    print("\n🔧 Correggi questi problemi prima di procedere con il training!")

## ✅ Step 3.2: Verifica Formato metadata.csv

Questa cella verifica che il metadata.csv sia correttamente formattato:
- **Formato**: `filename|transcription`
- **Separatore**: pipe `|`
- **Encoding**: UTF-8
- **Corrispondenza**: tutti i file citati esistono in wavs/

In [ ]:
# Verifica COMPLETA di tutti i file WAV
import wave
import os
from pathlib import Path

print("🔍 Verifica formato audio in corso...\n")

wavs_dir = Path(f"{DATASET_DIR}/wavs")
wav_files = list(wavs_dir.glob("*.wav"))

# Contatori
total_files = len(wav_files)
correct_files = 0
errors = []

# Requisiti Piper
REQUIRED_SAMPLE_RATE = 16000
REQUIRED_CHANNELS = 1  # Mono
REQUIRED_SAMPWIDTH = 2  # 16-bit = 2 bytes

print(f"📊 Analizzando {total_files} file WAV...\n")

for i, wav_path in enumerate(wav_files, 1):
    try:
        with wave.open(str(wav_path), 'rb') as wav:
            sample_rate = wav.getframerate()
            channels = wav.getnchannels()
            sampwidth = wav.getsampwidth()
            
            # Verifica requisiti
            is_correct = (
                sample_rate == REQUIRED_SAMPLE_RATE and
                channels == REQUIRED_CHANNELS and
                sampwidth == REQUIRED_SAMPWIDTH
            )
            
            if is_correct:
                correct_files += 1
            else:
                error_msg = f"{wav_path.name}: "
                issues = []
                
                if sample_rate != REQUIRED_SAMPLE_RATE:
                    issues.append(f"SR={sample_rate}Hz (richiesto {REQUIRED_SAMPLE_RATE}Hz)")
                if channels != REQUIRED_CHANNELS:
                    issues.append(f"Canali={channels} (richiesto {REQUIRED_CHANNELS})")
                if sampwidth != REQUIRED_SAMPWIDTH:
                    issues.append(f"Bit={sampwidth*8}-bit (richiesto 16-bit)")
                
                error_msg += ", ".join(issues)
                errors.append(error_msg)
    
    except Exception as e:
        errors.append(f"{wav_path.name}: ERRORE - {str(e)}")
    
    # Progress bar
    if i % 100 == 0 or i == total_files:
        print(f"   Progresso: {i}/{total_files} ({i*100//total_files}%)")

# Risultati
print("\n" + "="*60)
print("📊 RISULTATI VERIFICA AUDIO:")
print("="*60)
print(f"✅ File corretti: {correct_files}/{total_files} ({correct_files*100//total_files}%)")
print(f"❌ File con problemi: {len(errors)}")

if errors:
    print("\n⚠️ PROBLEMI RILEVATI:")
    print("-"*60)
    for error in errors[:10]:  # Mostra primi 10
        print(f"   • {error}")
    
    if len(errors) > 10:
        print(f"   ... e altri {len(errors)-10} file con problemi")
    
    print("\n💡 SOLUZIONE: Converti i file usando ffmpeg:")
    print("   for f in wavs/*.wav; do")
    print(f"       ffmpeg -i \"$f\" -ar {REQUIRED_SAMPLE_RATE} -ac {REQUIRED_CHANNELS} -sample_fmt s16 \"converted/${{f##*/}}\"")
    print("   done")
else:
    print("\n🎉 PERFETTO! Tutti i file audio sono nel formato corretto!")
    print(f"   ✓ Sample rate: {REQUIRED_SAMPLE_RATE} Hz")
    print(f"   ✓ Canali: Mono")
    print(f"   ✓ Bit depth: 16-bit")

## ✅ Step 3.1: Verifica Rigorosa Formato Audio

Questa cella verifica che **TUTTI** i file WAV rispettino i requisiti Piper:
- **Sample rate**: 22050 Hz
- **Canali**: Mono (1 canale)
- **Bit depth**: 16-bit

## 🎛️ Step 4: Configurazione Training

In [ ]:
# Crea file di configurazione
import json

config = {
    "audio": {
        "sample_rate": 16000,
        "max_wav_value": 32767.0,
        "filter_length": 1024,
        "hop_length": 256,
        "win_length": 1024
    },
    "model": {
        "name": "vits",
        "hidden_channels": 192,
        "inter_channels": 192,
        "filter_channels": 768,
        "n_heads": 2,
        "n_layers": 6,
        "kernel_size": 3,
        "p_dropout": 0.1
    },
    "training": {
        "epochs": 10000,
        "learning_rate": 0.0002,
        "batch_size": 16,
        "log_interval": 100,
        "save_interval": 1000,
        "num_workers": 4
    },
    "dataset": {
        "path": DATASET_DIR,
        "text_cleaners": ["english_cleaners"],  # ⚠️ Modifica per la tua lingua
        "language": "en-us"  # ⚠️ Modifica codice lingua
    }
}

# Salva configurazione
config_path = f"{OUTPUT_DIR}/config.json"
with open(config_path, 'w') as f:
    json.dump(config, f, indent=2)

print(f"✅ Configurazione salvata in {config_path}")
print("\n📝 Parametri principali:")
print(f"   - Epochs: {config['training']['epochs']}")
print(f"   - Batch size: {config['training']['batch_size']}")
print(f"   - Learning rate: {config['training']['learning_rate']}")

## 🚀 Step 5: Avvio Training

In [ ]:
# Preprocessing del dataset
print("🔄 Preprocessing dataset...")

!python /content/piper/src/python/piper_train/preprocess.py \
    --input-dir {DATASET_DIR} \
    --output-dir {OUTPUT_DIR}/preprocessed \
    --language {config['dataset']['language']} \
    --sample-rate {config['audio']['sample_rate']}

print("\n✅ Preprocessing completato!")

In [ ]:
# Training (questo richiederà diverse ore)
print("🎯 Avvio training... (questo richiederà 12-16 ore)")
print("💡 Puoi monitorare i progressi nel log sotto.\n")

%cd /content/piper/src/python

!python -m piper_train \
    --dataset-dir {OUTPUT_DIR}/preprocessed \
    --output-dir {OUTPUT_DIR}/checkpoints \
    --config {config_path} \
    --restore-checkpoint  # Riprende da ultimo checkpoint se interrotto

%cd /content
print("\n🎉 Training completato!")

## 📊 Step 6: Monitoraggio Training (Opzionale)

In [ ]:
# Visualizza ultimi checkpoint
import glob

checkpoints = sorted(glob.glob(f"{OUTPUT_DIR}/checkpoints/*.pt"))
print(f"📁 Checkpoint trovati: {len(checkpoints)}\n")

for ckpt in checkpoints[-5:]:  # Ultimi 5
    size_mb = os.path.getsize(ckpt) / (1024*1024)
    print(f"   {os.path.basename(ckpt)} ({size_mb:.1f}MB)")

## 🎵 Step 7: Export Modello Finale

In [ ]:
# Converti checkpoint PyTorch in ONNX (formato Piper)
print("📦 Export modello in formato ONNX...")

# Trova ultimo checkpoint
latest_checkpoint = sorted(glob.glob(f"{OUTPUT_DIR}/checkpoints/*.pt"))[-1]
print(f"   Usando: {os.path.basename(latest_checkpoint)}")

%cd /content/piper/src/python

!python -m piper_train.export_onnx \
    {latest_checkpoint} \
    {OUTPUT_DIR}/model.onnx

%cd /content
print("\n✅ Modello esportato: model.onnx")

## 🧪 Step 8: Test Modello

In [ ]:
# Test sintetizzazione
from IPython.display import Audio
import subprocess

test_text = "Questo è un test del modello addestrato."  # ⚠️ Modifica testo
output_wav = f"{OUTPUT_DIR}/test_output.wav"

print(f"🎤 Sintetizzo: '{test_text}'...\n")

# Usa Piper per generare audio
cmd = [
    "piper",
    "--model", f"{OUTPUT_DIR}/model.onnx",
    "--output_file", output_wav
]

subprocess.run(cmd, input=test_text.encode('utf-8'))

print("✅ Audio generato! Ascolta sotto:\n")
Audio(output_wav)

## 💾 Step 9: Download Modello

In [ ]:
# Crea archivio con modello e config
import shutil

print("📦 Creazione archivio finale...\n")

# Copia file necessari
model_package = f"{OUTPUT_DIR}/my_piper_model"
!mkdir -p {model_package}

shutil.copy(f"{OUTPUT_DIR}/model.onnx", model_package)
shutil.copy(config_path, model_package)

# Crea archivio ZIP
shutil.make_archive(model_package, 'zip', model_package)

print(f"✅ Modello pronto per il download!")
print(f"   Percorso: {model_package}.zip\n")

# Download
from google.colab import files
files.download(f"{model_package}.zip")

print("\n🎉 Training completato con successo!")

## 📝 Note Finali

### Prossimi Passi:
1. **Fine-Tuning**: Puoi ora usare questo modello come base per ulteriore fine-tuning
2. **Testing**: Prova il modello con frasi diverse per valutare la qualità
3. **Ottimizzazione**: Se i risultati non sono ottimali:
   - Aumenta il dataset (più audio = migliori risultati)
   - Aumenta epochs
   - Modifica learning rate

### Troubleshooting:
- **OOM (Out of Memory)**: Riduci batch_size nel config
- **Audio distorto**: Verifica che tutti i WAV siano 22050Hz mono
- **Training lento**: Assicurati di usare GPU T4 o superiore

### Risorse:
- 📖 [Piper Documentation](https://github.com/rhasspy/piper)
- 💬 [Piper Discussion](https://github.com/rhasspy/piper/discussions)

---

**Buon training! 🚀**